##### Convention
# Notebooks should be easy to use

`param` exports `Parameterize`, a special function that will parameterize existing notebooks.  Any single target assignment that 
can be literally evaluated is a notebook parameter.

In [1]:
    try:
        from .rites import AST, docify, Partial
    except:
        from rites import AST, docify, Partial

    from types import ModuleType
    import sys
    from importlib import reload
    from dataclasses import dataclass, field

In [2]:
    from ast import NodeTransformer, parse, Assign, literal_eval, dump, fix_missing_locations, Str

In [3]:
    def vars_to_sig(vars):
        from inspect import Parameter, Signature
        return Signature([Parameter(str, Parameter.KEYWORD_ONLY, default = vars[str]) for str in vars])

In [4]:
    from collections import OrderedDict
    @dataclass
    class FreeStatement(NodeTransformer):
        params: list = field(default_factory=list)
        globals: dict = field(default_factory=dict)
        def visit_FunctionDef(free, node): return node
        visit_ClassDef = visit_FunctionDef
                
        def visit_Assign(free, node):
            if len(node.targets):
                try:
                    if not getattr(node.targets[0], 'id', '_').startswith('_'):
                        free.globals[node.targets[0].id] = literal_eval(node.value)
                        return 
                except: ...
            return node
                
        def __call__(free, nodes):
            ast = free.visit(nodes)
            return free.globals, fix_missing_locations(ast)

In [5]:
    from inspect import signature

In [6]:
    from pathlib import Path

In [7]:
    from IPython.utils.capture import capture_output

In [8]:
    @dataclass(repr=False)
    class Parameterize:
        __file__: str = None
        __ast__: any = None
        __variables__: any = field(default_factory=dict)
        __notebook__: dict = field(default_factory=dict)
        def __post_init__(Parameterize):
            Parameterize.__name__ = Path(Parameterize.__file__).stem
            with open(Parameterize.__file__) as f: 
                Parameterize.__notebook__ = __import__('nbformat').read(f, 4)       
            with capture_output() as output:
                Parameterize.__variables__, Parameterize.__ast__ = \
                    FreeStatement()(AST().from_notebook_node(Parameterize.__notebook__))
            Parameterize.__output__ = output
            Parameterize.__signature__ = vars_to_sig(Parameterize.__variables__)
            Parameterize.__doc__ = docify(Parameterize.__notebook__)

        def __call__(Parameterize, **dict):
            Parameterize = __import__('copy').copy(Parameterize)
            Parameterize.__dict__.update(Parameterize.__variables__)
            Parameterize.__dict__.update(dict)
            exec(AST().compile(Parameterize.__ast__, Parameterize.__file__), *[Parameterize.__dict__]*2)
            return Parameterize
        
        def interact(Parameterize): 
            return __import__('ipywidgets').interact(Parameterize)

In [9]:
    __test__ = dict(
        self="""
        >>> f = Parameterize(globals().get('__file__', 'param.ipynb'))
        >>> assert f().param == 'xyz'
        >>> assert f(param=10).param is 10
        """
        ,)

In [10]:
    if __name__ == '__main__':
        __import__('doctest').testmod(verbose=1)
        f = Parameterize(globals().get('__file__', 'param.ipynb'))
    else:
        param = 'xyz'

Trying:
    f = Parameterize(globals().get('__file__', 'param.ipynb'))
Expecting nothing
ok
Trying:
    assert f().param == 'xyz'
Expecting nothing
ok
Trying:
    assert f(param=10).param is 10
Expecting nothing
ok
10 items had no tests:
    __main__
    __main__.FreeStatement
    __main__.FreeStatement.__call__
    __main__.FreeStatement.visit_Assign
    __main__.FreeStatement.visit_FunctionDef
    __main__.Parameterize
    __main__.Parameterize.__call__
    __main__.Parameterize.__post_init__
    __main__.Parameterize.interact
    __main__.vars_to_sig
1 items passed all tests:
   3 tests in __main__.__test__.self
3 tests in 11 items.
3 passed and 0 failed.
Test passed.
